## Task C: Logistic Regression using Transfer Learning

- Only 50 iterations of the Logistic Regression functions is run with the default parameters left unchanged. Tweaking these parameters might lead to better accuracies

In [1]:
from zipfile import ZipFile
import numpy as np
import mxnet as mx

display_step=10

/home/suyash/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class DataLoader(object):
    DIR=None
    import os
    print(os.getcwd())
    
    def __init__(self):
        self.DIR = './'
    
    # Returns images and labels corresponding for training and testing. Default mode is train. 
    # For retrieving test data pass mode as 'test' in function call.
    def load_data(self, mode = 'train'):
        label_filename = mode + '_labels'
        image_filename = mode + '_images'
        label_zip = self.DIR + label_filename + '.zip'
        image_zip = self.DIR + image_filename + '.zip'
        with ZipFile(label_zip, 'r') as lblzip:
            labels = np.frombuffer(lblzip.read(label_filename), dtype=np.uint8, offset=8)
        with ZipFile(image_zip, 'r') as imgzip:
            images = np.frombuffer(imgzip.read(image_filename), dtype=np.uint8, offset=16).reshape(len(labels), 784)
        return images, labels
    
    
    def create_batches(self,train_size):
        '''
        Handles the train-validation split 
        Params:
            train_size: ratio of train_size : float
        Returns:
            images, labels, validation images, validation labels in that order
        '''
        img,labels=self.load_data()
        n=labels.shape[0]
        #split into train and validation data
        validation_img=img[int(train_size*n):]
        img=img[:int(train_size*n)]
        validation_labels=labels[int(train_size*n):]
        labels=labels[:int(train_size*n)]
        return img, labels, validation_img, validation_labels        

/media/suyash/New Volume/IIT KGP/DeepLearning


In [3]:
data_loader=DataLoader()
data,label=data_loader.load_data(mode='train')
batch_size=data.shape[0]
train_iter = mx.io.NDArrayIter(data, label, batch_size, shuffle=True)

### Loading Network 2 completely

In [4]:
data_new = mx.symbol.Variable('data')
fc1_new  = mx.symbol.FullyConnected(data = data_new, num_hidden=1024,name='fc1')
act1_new = mx.symbol.Activation(data = fc1_new, act_type="relu",name='act1')
fc2_new  = mx.symbol.FullyConnected(data = act1_new, num_hidden = 512,name='fc2')
act2_new = mx.symbol.Activation(data = fc2_new, act_type="relu",name='act2')
fc3_new  = mx.symbol.FullyConnected(data = act2_new, num_hidden = 256,name='fc3')
act3_new = mx.symbol.Activation(data = fc3_new, act_type="relu",name='act3')

#output layer
fc4_new  = mx.symbol.FullyConnected(data = act3_new, num_hidden=10,name='fc4')
mlp_new = mx.symbol.SoftmaxOutput(data = fc4_new,name='softmax')
mod_new = mx.mod.Module(symbol=mlp_new,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])
mod_new.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
mod_new.load_params('network2_new')

In [11]:
sym_new=mlp_new.get_internals()['fc1_output']

In [12]:
mod_trans = mx.mod.Module(symbol=sym_new, context=mx.cpu(),data_names=['data'],label_names=None)
mod_trans.bind(for_training=False, data_shapes=train_iter.provide_data)
arg_params, aux_params = mod_new.get_params()
mod_trans.set_params(arg_params, aux_params)

In [13]:
train_iter.reset()
for batch in train_iter:
    mod_trans.forward(batch)
    out_trans=mod_trans.get_outputs()[0].asnumpy()
print(out_trans.shape)

(60000, 1024)


In [15]:
# obtaining test image transforms:
test_img,test_labels=data_loader.load_data(mode='test')
test_iter = mx.io.NDArrayIter(test_img,batch_size=test_img.shape[0])
test_iter.reset()
for batch in test_iter:
    mod_trans.forward(batch)
    out_trans_test=mod_trans.get_outputs()[0].asnumpy()
print(out_trans_test.shape)

(10000, 1024)


### Logistic Regression with data transformed to different layers

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle
clf=LogisticRegression(max_iter=50,solver='lbfgs')

In [16]:
clf.fit(out_trans,label)
y_predicted=clf.predict(out_trans_test)
accuracy=accuracy_score(test_labels,y_predicted)
print("Accuracy on classification task using features from neural net: ", accuracy)
pickle.dump(clf,open('layer1','wb'))

Accuracy on classification task using features from neural net:  0.1251


In [90]:
sym_new=mlp_new.get_internals()['fc2_output']
mod_trans = mx.mod.Module(symbol=sym_new, context=mx.cpu(),data_names=['data'],label_names=None)
mod_trans.bind(for_training=False, data_shapes=train_iter.provide_data)
arg_params, aux_params = mod_new.get_params()
mod_trans.set_params(arg_params, aux_params)
train_iter.reset()
for batch in train_iter:
    mod_trans.forward(batch)
    out_trans=mod_trans.get_outputs()[0].asnumpy()
print (out_trans.shape)
test_iter.reset()
for batch in test_iter:
    mod_trans.forward(batch)
    out_trans_test=mod_trans.get_outputs()[0].asnumpy()

(60000, 512)


In [91]:
clf.fit(out_trans,label)
y_predicted=clf.predict(out_trans_test)
accuracy=accuracy_score(test_labels,y_predicted)
print("Accuracy on classification task using features from second layer: ", accuracy)
pickle.dump(clf,open('layer2','wb'))

Accuracy on classification task using features from second layer:  0.0983


In [17]:
sym_new=mlp_new.get_internals()['fc3_output']
mod_trans = mx.mod.Module(symbol=sym_new, context=mx.cpu(),data_names=['data'],label_names=None)
mod_trans.bind(for_training=False, data_shapes=train_iter.provide_data)
arg_params, aux_params = mod_new.get_params()
mod_trans.set_params(arg_params, aux_params)
train_iter.reset()
for batch in train_iter:
    mod_trans.forward(batch)
    out_trans=mod_trans.get_outputs()[0].asnumpy()
print (out_trans.shape)
test_iter.reset()
for batch in test_iter:
    mod_trans.forward(batch)
    out_trans_test=mod_trans.get_outputs()[0].asnumpy()
clf.fit(out_trans,label)
y_predicted=clf.predict(out_trans_test)
accuracy=accuracy_score(test_labels,y_predicted)
print("Accuracy on classification task using features from third layer: ", accuracy)
pickle.dump(clf,open('laye3','wb'))

(60000, 256)
Accuracy on classification task using features from third layer:  0.0847
